# Préparation des Données pour ADAN Trading Bot

Ce notebook permet de générer les jeux de données nécessaires à l'entraînement du modèle ADAN.

In [1]:
import pandas as pd
import numpy as np
import yfinance as yf
import pandas_ta as ta
from pathlib import Path
from tqdm.auto import tqdm
import warnings
warnings.filterwarnings('ignore')

# Configuration
PAIRS = ['BTC-USD', 'ETH-USD', 'SOL-USD', 'XRP-USD', 'ADA-USD']
TIMEFRAMES = ['5m', '1h', '4h']
PERIOD = '2y'  # Période de données à télécharger
DATA_DIR = Path('../data')
RAW_DIR = DATA_DIR / 'raw'
PROCESSED_DIR = DATA_DIR / 'processed'
INDICATORS_DIR = PROCESSED_DIR / 'indicators'

# Création des répertoires
for dir_path in [RAW_DIR, PROCESSED_DIR, INDICATORS_DIR]:
    dir_path.mkdir(parents=True, exist_ok=True)

def download_data(pair: str, interval: str, period: str) -> pd.DataFrame:
    """Télécharge les données OHLCV depuis Yahoo Finance ou charge depuis CSV si disponible"""
    csv_file_path = RAW_DIR / f"{pair.replace('-', '')}_{interval}.csv"

    if csv_file_path.exists():
        print(f"Chargement des données pour {pair} {interval} depuis {csv_file_path}...")
        try:
            df = pd.read_csv(csv_file_path, index_col='Date', parse_dates=True)
            if df.empty:
                print(f"Fichier CSV vide pour {pair} {interval}")
                # Fallback to download if CSV is empty
                pass
            else:
                return df
        except Exception as e:
            print(f"Erreur lors du chargement du CSV pour {pair} {interval}: {e}")
            # Fallback to download if CSV loading fails
            pass

    print(f"Téléchargement des données pour {pair} {interval} depuis Yahoo Finance...")
    try:
        df = yf.download(
            pair,
            period=period,
            interval=interval,
            progress=True,  # Activate progression
            show_errors=True # Show errors
        )
        if df.empty:
            print(f"Aucune donnée pour {pair} {interval}")
            return None
        
        # Save downloaded data to CSV for future use
        df.to_csv(csv_file_path)
        print(f"Données téléchargées et sauvegardées en CSV pour {pair} {interval}")
        return df
    except Exception as e:
        print(f"Erreur lors du téléchargement de {pair} {interval}: {e}")
        return None

def calculate_indicators(df: pd.DataFrame, interval: str) -> pd.DataFrame:
    """Calcule les indicateurs techniques en fonction du timeframe"""
    if df is None or df.empty:
        return None

    # Copie pour éviter les modifications sur place
    df = df.copy()

    # Calcul des indicateurs communs
    df['returns'] = df['Close'].pct_change()

    # Indicateurs spécifiques au timeframe
    if interval == '5m':
        # Indicateurs pour le scalping
        df.ta.rsi(length=14, append=True)
        df.ta.stoch(k=14, d=3, smooth_k=3, append=True)
        df.ta.cci(length=20, append=True)
        df.ta.roc(length=9, append=True)
        df.ta.mfi(length=14, append=True)
        df.ta.ema(length=5, append=True)
        df.ta.ema(length=20, append=True)
        df.ta.supertrend(length=14, multiplier=2.0, append=True)
        df.ta.psar(af0=0.02, af=0.2, max_af=0.2, append=True)

    elif interval == '1h':
        # Indicateurs pour le swing trading
        df.ta.rsi(length=14, append=True)
        df.ta.macd(fast=12, slow=26, signal=9, append=True)
        df.ta.cci(length=20, append=True)
        df.ta.mfi(length=14, append=True)
        df.ta.ema(length=50, append=True)
        df.ta.ema(length=100, append=True)
        df.ta.sma(length=200, append=True)
        df.ta.ichimoku(tenkan=9, kijun=26, senkou=52, append=True)
        df.ta.psar(af0=0.02, af=0.2, max_af=0.2, append=True)

    elif interval == '4h':
        # Indicateurs pour le position trading
        df.ta.rsi(length=14, append=True)
        df.ta.macd(fast=12, slow=26, signal=9, append=True)
        df.ta.cci(length=20, append=True)
        df.ta.mfi(length=14, append=True)
        df.ta.sma(length=200, append=True)
        df.ta.ema(length=50, append=True)
        df.ta.ichimoku(tenkan=9, kijun=26, senkou=52, append=True)
        df.ta.supertrend(length=14, multiplier=3.0, append=True)
        df.ta.psar(af0=0.02, af=0.2, max_af=0.2, append=True)

    # Indicateurs communs à tous les timeframes
    df.ta.atr(length=14, append=True)
    df.ta.bbands(length=20, std=2.0, append=True)
    df.ta.obv(append=True)

    if interval in ['1h', '4h']:
        df.ta.vwap(anchor='D', append=True)  # VWAP journalier
    if interval == '4h':
        df.ta.vwap(anchor='W', append=True)  # VWAP hebdomadaire

    return df

def process_pair(pair: str):
    """Traite une paire de trading pour tous les timeframes"""
    pair_dir = INDICATORS_DIR / pair.replace('-', '')
    pair_dir.mkdir(exist_ok=True)

    for tf in TIMEFRAMES:
        print(f"Traitement de {pair} - {tf}...")

        # Téléchargement des données
        df = download_data(pair, tf, PERIOD)
        if df is None or df.empty:
            continue

        # Calcul des indicateurs
        df = calculate_indicators(df, tf)
        if df is None or df.empty:
            continue

        # Nettoyage des données
        df = df.dropna()

        # Division en ensembles train/val/test (70%/15%/15%)
        train_size = int(len(df) * 0.7)
        val_size = int(len(df) * 0.15)

        train_df = df.iloc[:train_size]
        val_df = df.iloc[train_size:train_size + val_size]
        test_df = df.iloc[train_size + val_size:]

        # Sauvegarde
        for split, split_df in [('train', train_df), ('val', val_df), ('test', test_df)]:
            save_dir = INDICATORS_DIR / split / pair.replace('-', '') / f'{tf}.parquet'
            save_dir.parent.mkdir(parents=True, exist_ok=True)
            split_df.to_parquet(save_dir)

        print(f"  ✓ Données sauvegardées pour {pair} - {tf}")

# Traitement de toutes les paires
for pair in tqdm(PAIRS, desc="Traitement des paires"):
    process_pair(pair)

print("Traitement terminé avec succès!")

Traitement des paires:   0%|          | 0/5 [00:00<?, ?it/s]

Traitement de BTC-USD - 5m...
Téléchargement des données pour BTC-USD 5m depuis Yahoo Finance...
Erreur lors du téléchargement de BTC-USD 5m: download() got an unexpected keyword argument 'show_errors'
Traitement de BTC-USD - 1h...
Téléchargement des données pour BTC-USD 1h depuis Yahoo Finance...
Erreur lors du téléchargement de BTC-USD 1h: download() got an unexpected keyword argument 'show_errors'
Traitement de BTC-USD - 4h...
Téléchargement des données pour BTC-USD 4h depuis Yahoo Finance...
Erreur lors du téléchargement de BTC-USD 4h: download() got an unexpected keyword argument 'show_errors'
Traitement de ETH-USD - 5m...
Téléchargement des données pour ETH-USD 5m depuis Yahoo Finance...
Erreur lors du téléchargement de ETH-USD 5m: download() got an unexpected keyword argument 'show_errors'
Traitement de ETH-USD - 1h...
Téléchargement des données pour ETH-USD 1h depuis Yahoo Finance...
Erreur lors du téléchargement de ETH-USD 1h: download() got an unexpected keyword argument 'sho